# Formula/Function

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import yfinance as yf
import pandas as pd
from datetime import datetime

##Black-Schole

In [2]:
import numpy as np
from scipy.stats import norm

class Black＿Scholes:
    def __init__(self, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility):
        self.spot_price = spot_price
        self.strike_price = strike_price
        self.dividend_yield = dividend_yield
        self.risk_free_rate = risk_free_rate
        self.time_to_maturity = time_to_maturity
        self.volatility = volatility

    def d1(self):
        d1 = (np.log(self.spot_price / self.strike_price) + (self.dividend_yield + (self.volatility ** 2) / 2) * self.time_to_maturity) / (self.volatility * np.sqrt(self.time_to_maturity))
        return d1

    def d2(self):
        d1 = self.d1()
        d2 = d1 - self.volatility * np.sqrt(self.time_to_maturity)
        return d2

    def call_option_price(self):
        d1 = self.d1()
        d2 = self.d2()
        call_option_price = self.spot_price * np.exp((self.dividend_yield - self.risk_free_rate) * self.time_to_maturity) * norm.cdf(d1) - self.strike_price * np.exp(-self.risk_free_rate * self.time_to_maturity) * norm.cdf(d2)
        return call_option_price

    def call_option_delta(self):
        d1 = self.d1()
        call_option_delta = np.exp((self.dividend_yield - self.risk_free_rate) * self.time_to_maturity) * norm.cdf(d1)
        return call_option_delta

    def put_option_price(self):
        d1 = self.d1()
        d2 = self.d2()
        put_option_price = self.strike_price * np.exp(-self.risk_free_rate * self.time_to_maturity) * norm.cdf(-d2) - self.spot_price * np.exp((self.dividend_yield - self.risk_free_rate) * self.time_to_maturity) * norm.cdf(-d1)
        return put_option_price

    def put_option_delta(self):
        d1 = self.d1()
        put_option_delta = -np.exp((self.dividend_yield - self.risk_free_rate) * self.time_to_maturity) * norm.cdf(-d1)
        return put_option_delta

##Binomial Tree

In [3]:
import numpy as np
import pandas as pd

class BinomialOptionPricing:
    def __init__(self, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps):
        self.spot_price = spot_price
        self.strike_price = strike_price
        self.dividend_yield = dividend_yield
        self.risk_free_rate = risk_free_rate
        self.time_to_maturity = time_to_maturity
        self.volatility = volatility
        self.num_steps = num_steps
        self.delta_t = time_to_maturity / num_steps
        self.u = np.exp(volatility * np.sqrt(self.delta_t))
        self.d = 1 / self.u
        self.r = np.exp(risk_free_rate * self.delta_t)
        self.b = np.exp(dividend_yield * self.delta_t)
        self.q = (self.b - self.d) / (self.u - self.d)
        self.y = self.generate_asset_price()

    def generate_asset_price(self):
        y = np.zeros((self.num_steps + 1, self.num_steps + 1))

        for i in range(self.num_steps + 1):
            for j in range(self.num_steps + 1):
                if i == 0 and j == 0:
                    y[i, j] = self.spot_price
                elif i == j:
                    y[i, j] = y[i-1, j-1] * self.d
                elif i < j:
                    y[i, j] = y[i, j-1] * self.u
                else:
                    y[i, j] = 0

        return y

    def generate_european_call(self):
        x = np.zeros((self.num_steps + 1, self.num_steps + 1))

        for i in range(self.num_steps + 1):
            for j in range(self.num_steps + 1):
                if j == self.num_steps:
                    x[i, j] = max(0, self.y[i, j] - self.strike_price)

        n = 1
        for k in range(self.num_steps + 1 - n):
            for i in range(self.num_steps - n + 1):
                for j in range(self.num_steps):
                    if j <= self.num_steps - 1 and i <= self.num_steps - 1 and i <= j:
                        x[i, j] = (self.q * x[i, j+1] + (1 - self.q) * x[i+1, j+1]) / self.r
            n += 1
        return x

    def generate_american_call(self):
        x = np.zeros((self.num_steps + 1, self.num_steps + 1))

        for i in range(self.num_steps + 1):
            for j in range(self.num_steps + 1):
                if j == self.num_steps:
                    x[i, j] = max(0, self.y[i, j] - self.strike_price)

        n = 1
        for k in range(self.num_steps + 1 - n):
            for i in range(self.num_steps - n + 1):
                for j in range(self.num_steps):
                    if j <= self.num_steps - 1 and i <= self.num_steps - 1 and i <= j:
                        x[i, j] = np.maximum(self.y[i, j] - self.strike_price, (self.q * x[i, j+1] + (1 - self.q) * x[i+1, j+1]) / self.r)

            n += 1
        return x

    def generate_european_put(self):
        x = np.zeros((self.num_steps + 1, self.num_steps + 1))

        for i in range(self.num_steps + 1):
            for j in range(self.num_steps + 1):
                if j == self.num_steps:
                    x[i, j] = max(0, self.strike_price - self.y[i, j])

        n = 1
        for k in range(self.num_steps + 1 - n):
            for i in range(self.num_steps - n + 1):
                for j in range(self.num_steps):
                    if j <= self.num_steps - 1 and i <= self.num_steps - 1 and i <= j:
                        x[i, j] = (self.q * x[i, j+1] + (1 - self.q) * x[i+1, j+1]) / self.r
            n += 1
        return x

    def generate_american_put(self):
        x = np.zeros((self.num_steps + 1, self.num_steps + 1))

        for i in range(self.num_steps + 1):
            for j in range(self.num_steps + 1):
                if j == self.num_steps:
                    x[i, j] = max(0, self.strike_price - self.y[i, j])

        n = 1
        for k in range(self.num_steps + 1 - n):
            for i in range(self.num_steps - n + 1):
                for j in range(self.num_steps):
                    if j <= self.num_steps - 1 and i <= self.num_steps - 1 and i <= j:
                        x[i, j] = np.maximum(self.strike_price - self.y[i, j], (self.q * x[i, j+1] + (1 - self.q) * x[i+1, j+1]) / self.r)

            n += 1
        return x

    def european_call_price(self):
      european_call_price = self.generate_european_call()
      return european_call_price[0, 0]

    def american_call_price(self):
      american_call_price = self.generate_american_call()
      return american_call_price[0, 0]

    def european_put_price(self):
      generate_european_put = self.generate_european_put()
      return generate_european_put[0, 0]

    def american_put_price(self):
      generate_american_put = self.generate_american_put()
      return generate_american_put[0, 0]


# Real Option Data

## Grab Data from Yahoo Finance

In [4]:
import yfinance as yf
import pandas as pd

ticker_symbol = 'SPY'
ticker = yf.Ticker(ticker_symbol)

# Retrieve available expiration dates for options
expiration_dates = ticker.options

# Convert expiration dates to a DataFrame for display
if expiration_dates:
    expiration_dates_df = pd.DataFrame(expiration_dates, columns=['Expire Dates'])
    print(expiration_dates_df)
else:
    print('No expiration dates available.')

print('Number of expiration dates:', len(expiration_dates))

   Expire Dates
0    2024-05-06
1    2024-05-07
2    2024-05-08
3    2024-05-09
4    2024-05-10
5    2024-05-13
6    2024-05-14
7    2024-05-15
8    2024-05-16
9    2024-05-17
10   2024-05-24
11   2024-05-31
12   2024-06-07
13   2024-06-14
14   2024-06-21
15   2024-06-28
16   2024-07-19
17   2024-07-31
18   2024-08-16
19   2024-08-30
20   2024-09-20
21   2024-09-30
22   2024-10-18
23   2024-10-31
24   2024-12-20
25   2024-12-31
26   2025-01-17
27   2025-03-21
28   2025-03-31
29   2025-06-20
30   2025-09-19
31   2025-12-19
32   2026-01-16
33   2026-12-18
Number of expiration dates: 34


In [5]:
import yfinance as yf
import pandas as pd

#expiration_date = '2025-03-21'  # Replace '2024-06-19' with your desired expiration date
expiration_date = expiration_dates[10]

# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)

# Retrieve the option chain for the specified expiration date
option_chain = ticker.option_chain(expiration_date)

# Convert the calls and puts data into DataFrames
calls_df = pd.DataFrame(option_chain.calls)
puts_df = pd.DataFrame(option_chain.puts)
expiration_date = pd.to_datetime(expiration_date)
expiration_date

print(f"Total Call Numbers is {len(calls_df)}")
print(f"Total Put Numbers is {len(puts_df)}")

Total Call Numbers is 90
Total Put Numbers is 82


In [6]:
#risk-free
import yfinance as yf
from datetime import datetime

# Define the ticker symbols for Treasury bonds
tickers = ["^IRX", "^FVX", "^TNX", "^TYX"]  # 6-month T-bill, 5-year, 10-year, 30-year

# Fetch data
Risk_Free_Rate = yf.download(tickers, start = datetime.today().date() - pd.Timedelta(days = 7), end=datetime.today().date(), interval="1wk")
# Filter only 'Adj Close' columns
adj_close_columns = [col for col in Risk_Free_Rate.columns if 'Adj Close' in col]
Risk_Free_Rate_adj_close = Risk_Free_Rate[adj_close_columns]
# Clean column names
Risk_Free_Rate_adj_close.columns = [col[1] for col in Risk_Free_Rate_adj_close.columns]
Risk_Free_Rate = Risk_Free_Rate_adj_close.reset_index()
Risk_Free_Rate = Risk_Free_Rate.rename(columns = {"^IRX": "TB13W", "^FVX": "TB5", "^TNX": "TB10", "^TYX": "TB30"})
Risk_Free_Rate["Date"] = pd.to_datetime(Risk_Free_Rate["Date"])
Risk_Free_Rate.iloc[:, 1:] = Risk_Free_Rate.iloc[:, 1:]/100

[*********************100%%**********************]  4 of 4 completed


In [7]:
Risk_Free = Risk_Free_Rate["TB13W"].tail(1).values[0]

In [8]:
difference = expiration_date - pd.Timestamp(datetime.today().date())
difference.days

18

In [9]:
import yfinance as yf

ticker = yf.Ticker(ticker_symbol)

# Retrieve historical data for the past 3 months
historical_data = ticker.history(period= '52wk', interval='1wk')
historical_data = historical_data.reset_index()
historical_data['Date'] = pd.to_datetime(historical_data['Date'])
historical_data['Date'] = historical_data['Date'].dt.strftime('%Y-%m-%d')
historical_data.head(3)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
0,2023-05-08,407.075089,408.622685,403.033608,405.714783,336006300,0.0,0.0,0.0
1,2023-05-15,406.335802,414.714469,404.374218,412.644440,400138800,0.0,0.0,0.0
2,2023-05-22,412.664169,414.763739,404.029203,414.024445,421134200,0.0,0.0,0.0


In [10]:
historical_data['Dividends_Yield'] = historical_data['Dividends']/historical_data['Close']
dividend_yield_rate = historical_data['Dividends_Yield'].mean() * 52
dividend_yield_rate

0.01461841453337612

In [11]:
historical_data['Return'] = historical_data['Close'].pct_change()
historical_data = historical_data.fillna(0)
historical_data.round(3).head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Dividends_Yield,Return
0,2023-05-08,407.075,408.623,403.034,405.715,336006300,0.0,0.0,0.0,0.0,0.000
1,2023-05-15,406.336,414.714,404.374,412.644,400138800,0.0,0.0,0.0,0.0,0.017
2,2023-05-22,412.664,414.764,404.029,414.024,421134200,0.0,0.0,0.0,0.0,0.003
3,2023-05-29,416.006,422.620,410.279,421.812,363259500,0.0,0.0,0.0,0.0,0.019
4,2023-06-05,422.167,425.824,419.742,423.763,362551300,0.0,0.0,0.0,0.0,0.005


In [12]:
volatility_past = historical_data['Return'].std()/np.sqrt(1/52) * np.sqrt(12)
volatility_past

0.4215755369991052

In [13]:
import yfinance as yf

# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)
# Retrieve the real-time price (most recent price)
real_time_data = ticker.history(period='1d')
# Extract the spot price (most recent closing price)
if not real_time_data.empty:
    spot_price = real_time_data['Close'].iloc[-1]
    print(f"Spot price for {ticker_symbol}: ${spot_price:.2f}")
else:
    print('No real-time data available.')

Spot price for SPY: $516.57


## Call Option

In [14]:
calls_df.head(3)

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,SPY240524C00395000,2024-04-29 14:48:21+00:00,395.0,116.41,122.42,122.78,0.0,0.0,2.0,5,0.661624,True,REGULAR,USD
1,SPY240524C00400000,2024-05-03 14:42:06+00:00,400.0,111.09,117.44,117.80,0.0,0.0,3.0,23,0.637699,True,REGULAR,USD
2,SPY240524C00405000,2024-04-29 13:58:48+00:00,405.0,106.45,112.47,112.82,0.0,0.0,15.0,16,0.614506,True,REGULAR,USD


In [15]:
calls_df = calls_df[['strike', 'lastPrice', 'bid', 'ask','impliedVolatility']]
calls_df.insert(0, 'Tickers', ticker_symbol)

In [16]:
calls_df['Spot Price'] = spot_price.round(2)

<ipython-input-16-0be5115fe159>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls_df['Spot Price'] = spot_price.round(2)


In [17]:
calls_df['Volatility_Past'] = volatility_past

In [18]:
calls_df_price = calls_df.copy()

In [19]:
calls_df_price.head(3)

,Tickers,strike,lastPrice,bid,ask,impliedVolatility,Spot Price,Volatility_Past
0,SPY,395.0,116.41,122.42,122.78,0.661624,516.57,0.421576
1,SPY,400.0,111.09,117.44,117.80,0.637699,516.57,0.421576
2,SPY,405.0,106.45,112.47,112.82,0.614506,516.57,0.421576


In [20]:
def calculate_bs_call_price(row, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility):
    bs = Black＿Scholes(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility)
    return bs.call_option_price()
def calculate_bt_call_eu_price(row, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps):
    bt = BinomialOptionPricing(spot_price, strike_price, dividend_yield,risk_free_rate, time_to_maturity, volatility, num_steps)
    return bt.european_call_price()
def calculate_bt_call_am_price(row, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps):
    bt = BinomialOptionPricing(spot_price, strike_price, dividend_yield,risk_free_rate, time_to_maturity, volatility, num_steps)
    return bt.american_call_price()

In [21]:
num_steps = 20

In [22]:
#pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [23]:
# Assuming calls_df is your DataFrame containing options data
calls_df_price['BS_Price'] = calls_df_price.apply(lambda row: calculate_bs_call_price(row, spot_price, row['strike'], Risk_Free - dividend_yield_rate, Risk_Free, difference.days/365, row['impliedVolatility']), axis=1)
calls_df_price['BS_Price'] = calls_df_price['BS_Price'].round(2)

# Assuming calls_df is your DataFrame containing options data
calls_df_price['BT_EU_Price'] = calls_df_price.apply(lambda row: calculate_bt_call_eu_price(row, spot_price, row['strike'], Risk_Free - dividend_yield_rate, Risk_Free, difference.days/365, row['impliedVolatility'], num_steps), axis=1)
calls_df_price['BT_EU_Price'] = calls_df_price['BT_EU_Price'].round(2)

# Assuming calls_df is your DataFrame containing options data
calls_df_price['BT_AM_Price'] = calls_df_price.apply(lambda row: calculate_bt_call_am_price(row, spot_price, row['strike'], Risk_Free - dividend_yield_rate, Risk_Free, difference.days/365, row['impliedVolatility'], num_steps), axis=1)
calls_df_price['BT_AM_Price'] = calls_df_price['BT_AM_Price'].round(2)

In [24]:
calls_df_price = calls_df_price[['Tickers','Spot Price', 'strike', 'lastPrice', 'bid', 'ask', 'impliedVolatility', 'Volatility_Past','BS_Price', 'BT_EU_Price', 'BT_AM_Price']]

In [25]:
calls_df_price.head()

,Tickers,Spot Price,strike,lastPrice,bid,ask,impliedVolatility,Volatility_Past,BS_Price,BT_EU_Price,BT_AM_Price
0,SPY,516.57,395.0,116.41,122.42,122.78,0.661624,0.421576,123.07,122.97,122.97
1,SPY,516.57,400.0,111.09,117.44,117.80,0.637699,0.421576,118.10,117.99,117.99
2,SPY,516.57,405.0,106.45,112.47,112.82,0.614506,0.421576,113.14,113.02,113.02
3,SPY,516.57,410.0,106.33,107.49,107.84,0.590580,0.421576,108.17,108.08,108.08
4,SPY,516.57,415.0,101.56,102.51,102.86,0.566899,0.421576,103.20,103.13,103.13


## Put Option

In [26]:
puts_df = puts_df[['strike', 'lastPrice', 'bid', 'ask','impliedVolatility']]
puts_df.insert(0, 'Tickers', ticker_symbol)
puts_df.head()

,Tickers,strike,lastPrice,bid,ask,impliedVolatility
0,SPY,395.0,0.04,0.03,0.04,0.427740
1,SPY,400.0,0.05,0.03,0.04,0.410162
2,SPY,405.0,0.04,0.04,0.05,0.400397
3,SPY,410.0,0.08,0.04,0.05,0.382819
4,SPY,415.0,0.05,0.05,0.06,0.371100


In [27]:
puts_df['Spot Price'] = spot_price.round(2)
puts_df['Volatility_Past'] = volatility_past
puts_df_price = puts_df.copy()
puts_df_price.head(3)

<ipython-input-27-eea48457bca4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  puts_df['Spot Price'] = spot_price.round(2)


,Tickers,strike,lastPrice,bid,ask,impliedVolatility,Spot Price,Volatility_Past
0,SPY,395.0,0.04,0.03,0.04,0.427740,516.57,0.421576
1,SPY,400.0,0.05,0.03,0.04,0.410162,516.57,0.421576
2,SPY,405.0,0.04,0.04,0.05,0.400397,516.57,0.421576


In [28]:
def calculate_bs_put_price(row, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility):
    bs = Black＿Scholes(spot_price, strike_price, Risk_Free - dividend_yield, risk_free_rate, time_to_maturity, volatility)
    return bs.put_option_price()
def calculate_bt_put_eu_price(row, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps):
    bt = BinomialOptionPricing(spot_price, strike_price, Risk_Free - dividend_yield,risk_free_rate, time_to_maturity, volatility, num_steps)
    return bt.european_put_price()
def calculate_bt_put_am_price(row, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps):
    bt = BinomialOptionPricing(spot_price, strike_price, Risk_Free - dividend_yield,risk_free_rate, time_to_maturity, volatility, num_steps)
    return bt.american_put_price()

In [29]:
num_steps = 40

In [30]:
# Assuming calls_df is your DataFrame containing options data
puts_df_price['BS_Price'] = puts_df_price.apply(lambda row: calculate_bs_put_price(row, spot_price, row['strike'], dividend_yield_rate, Risk_Free, difference.days/365, row['impliedVolatility']), axis=1)
puts_df_price['BS_Price'] = puts_df_price['BS_Price'].round(2)

# Assuming calls_df is your DataFrame containing options data
puts_df_price['BT_EU_Price'] = puts_df_price.apply(lambda row: calculate_bt_put_eu_price(row, spot_price, row['strike'], dividend_yield_rate, Risk_Free, difference.days/365, row['impliedVolatility'], num_steps), axis=1)
puts_df_price['BT_EU_Price'] = puts_df_price['BT_EU_Price'].round(2)

# Assuming calls_df is your DataFrame containing options data
puts_df_price['BT_AM_Price'] = puts_df_price.apply(lambda row: calculate_bt_put_am_price(row, spot_price, row['strike'], dividend_yield_rate, Risk_Free, difference.days/365, row['impliedVolatility'], num_steps), axis=1)
puts_df_price['BT_AM_Price'] = puts_df_price['BT_AM_Price'].round(2)

In [31]:
puts_df_price = puts_df_price[['Tickers','Spot Price', 'strike', 'lastPrice', 'bid', 'ask', 'impliedVolatility', 'Volatility_Past','BS_Price', 'BT_EU_Price', 'BT_AM_Price']]

In [32]:
puts_df_price.tail()

,Tickers,Spot Price,strike,lastPrice,bid,ask,impliedVolatility,Volatility_Past,BS_Price,BT_EU_Price,BT_AM_Price
77,SPY,516.57,540.0,36.55,23.55,23.89,0.129892,0.421576,22.88,22.87,23.47
78,SPY,516.57,545.0,30.45,28.55,28.89,0.150399,0.421576,27.86,27.84,28.47
79,SPY,516.57,550.0,34.30,33.55,33.89,0.170052,0.421576,32.84,32.83,33.47
80,SPY,516.57,555.0,40.80,38.55,38.89,0.189217,0.421576,37.82,37.82,38.47
81,SPY,516.57,565.0,46.04,52.84,53.14,0.415045,0.421576,51.88,51.81,52.09


# Run Option Value

## Black-Schole

### Call Option

In [33]:
import numpy as np
import pandas as pd
from scipy.stats import norm
# Example usage:
spot_price = 516.57   # Current stock price
strike_price = 155.0   # Option strike price
time_to_maturity = difference.days/365  # Time to option expiration in years
volatility = 0.421576 #np.sqrt(0.1)  # Volatility of the underlying stock
risk_free_rate = Risk_Free # Risk-free interest rate
dividend =  dividend_yield_rate
dividend_yield =  risk_free_rate - dividend

In [34]:
Black_Schole_Option = Black＿Scholes(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility)

In [35]:
print(f"d1 is {Black_Schole_Option.d1():.3f}\nd2 is {Black_Schole_Option.d2():.3f}")
print(f"N(d1) is {norm.cdf(Black_Schole_Option.d1()):.3f}\nN(d2) is {norm.cdf(Black_Schole_Option.d2()):.3f}")
print(f"Call Value is {Black_Schole_Option.call_option_price():.3f}\nCall Delta is {Black_Schole_Option.call_option_delta():.3f}")

d1 is 12.925
d2 is 12.831
N(d1) is 1.000
N(d2) is 1.000
Call Value is 361.597
Call Delta is 0.999


In [36]:
initial_guess = 0.7
tolerance = 1e-3
max_iterations = 100

In [37]:
def implied_call_volatility(call_price, initial_guess=initial_guess, tolerance=tolerance, max_iterations=max_iterations):
    def black_scholes_call_price(volatility):
        Black_Schole_Option.volatility = volatility
        return Black_Schole_Option.call_option_price()

    # Newton-Raphson method
    vol = initial_guess
    for i in range(max_iterations):
        option_price = black_scholes_call_price(vol)
        vega = Black_Schole_Option.call_option_delta() * Black_Schole_Option.spot_price * np.exp(-Black_Schole_Option.dividend_yield * Black_Schole_Option.time_to_maturity) / np.sqrt(2 * np.pi * Black_Schole_Option.time_to_maturity)

        if np.abs(option_price - call_price) < tolerance:
            return vol

        vol -= (option_price - call_price) / vega

    return np.nan  # If not found within tolerance

call_price = 345.25
implied_vol = implied_call_volatility(call_price)
rounded_implied_vol = round(float(implied_vol), 3)
print("Implied Volatility Call:", rounded_implied_vol)

Implied Volatility Call: nan


<ipython-input-37-f129df46105d>:15: RuntimeWarning: divide by zero encountered in scalar divide
  vol -= (option_price - call_price) / vega
<ipython-input-2-60f38cd9a963>:14: RuntimeWarning: invalid value encountered in scalar divide
  d1 = (np.log(self.spot_price / self.strike_price) + (self.dividend_yield + (self.volatility ** 2) / 2) * self.time_to_maturity) / (self.volatility * np.sqrt(self.time_to_maturity))


### Put Option

In [38]:
import numpy as np
import pandas as pd
from scipy.stats import norm
# Example usage:
spot_price = 516.57   # Current stock price
strike_price = 540.0   # Option strike price
time_to_maturity = difference.days/365  # Time to option expiration in years
volatility = 0.129892 #np.sqrt(0.1)  # Volatility of the underlying stock
risk_free_rate = Risk_Free # Risk-free interest rate
dividend =  dividend_yield_rate
dividend_yield =  risk_free_rate - dividend

In [39]:
Black_Schole_Option = Black＿Scholes(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility)

In [40]:
print(f"d1 is {Black_Schole_Option.d1():.3f}\nd2 is {Black_Schole_Option.d2():.3f}")
print(f"N(d1) is {norm.cdf(Black_Schole_Option.d1()):.3f}\nN(d2) is {norm.cdf(Black_Schole_Option.d2()):.3f}")
print(f"Put Value is {Black_Schole_Option.put_option_price():.3f}\nPut Delta is {Black_Schole_Option.put_option_delta():.3f}")

d1 is -1.459
d2 is -1.488
N(d1) is 0.072
N(d2) is 0.068
Put Value is 22.884
Put Delta is -0.927


In [41]:
initial_guess = 0.7
tolerance = 1e-3
max_iterations = 100

In [42]:
def implied_put_volatility(put_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, initial_guess=initial_guess, tolerance=tolerance, max_iterations=max_iterations):
    def black_scholes_put_price(volatility):
        bs = Black_Scholes(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility)
        return bs.put_option_price()

    # Newton-Raphson method
    vol = initial_guess
    for _ in range(max_iterations):
        option_price = black_scholes_put_price(vol)
        vega = (black_scholes_put_price(vol * 1.001) - black_scholes_put_price(vol)) / (0.001 * vol)

        if abs(option_price - put_price) < tolerance:
            return vol

        vol -= (option_price - put_price) / vega

    return np.nan  # If not found within tolerance

put_price = 36.55
implied_vol = implied_put_volatility(put_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity)
rounded_implied_vol = round(float(implied_vol), 3)
print("Implied Volatility Put:", rounded_implied_vol)


Implied Volatility Put: 0.507


## Binomial Tree

### Call Option

In [43]:
import numpy as np

spot_price = 516.57   # Current stock price
strike_price = 405.0   # Option strike price
time_to_maturity = difference.days/365  # Time to option expiration in years
volatility = 0.614506 #np.sqrt(0.1)  # Volatility of the underlying stock
risk_free_rate = Risk_Free # Risk-free interest rate
dividend =  dividend_yield_rate
dividend_yield =  risk_free_rate - dividend

num_steps = 20

binomial_tree_option = BinomialOptionPricing(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps)

In [44]:
Asset_Price_df = pd.DataFrame(binomial_tree_option.generate_asset_price())
Asset_Price_df = Asset_Price_df.round(2)
#Asset_Price_df.replace(0, '', inplace=True)
#Asset_Price_df.head(num_steps+1)
Asset_Price_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,516.57,532.58,549.08,566.09,583.63,601.71,620.36,639.58,659.40,679.83,...,722.61,745.00,768.08,791.88,816.41,841.71,867.79,894.68,922.40,950.98
1,0.00,501.05,516.57,532.58,549.08,566.09,583.63,601.71,620.36,639.58,...,679.83,700.89,722.61,745.00,768.08,791.88,816.41,841.71,867.79,894.68
2,0.00,0.00,485.99,501.05,516.57,532.58,549.08,566.09,583.63,601.71,...,639.58,659.40,679.83,700.89,722.61,745.00,768.08,791.88,816.41,841.71
3,0.00,0.00,0.00,471.38,485.99,501.05,516.57,532.58,549.08,566.09,...,601.71,620.36,639.58,659.40,679.83,700.89,722.61,745.00,768.08,791.88
4,0.00,0.00,0.00,0.00,457.22,471.38,485.99,501.05,516.57,532.58,...,566.09,583.63,601.71,620.36,639.58,659.40,679.83,700.89,722.61,745.00


In [45]:
euro_call_df = pd.DataFrame(binomial_tree_option.generate_european_call())
euro_call_df = euro_call_df.round(2)
#euro_call_df.replace(0, '', inplace=True)
euro_call_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,113.02,128.55,144.79,161.67,179.14,197.17,215.78,234.96,254.74,275.13,...,317.84,340.20,363.25,387.02,411.53,436.80,462.85,489.72,517.42,545.98
1,0.00,97.89,112.73,128.36,144.67,161.58,179.07,197.11,215.72,234.90,...,275.08,296.11,317.79,340.15,363.20,386.97,411.48,436.75,462.81,489.68
2,0.00,0.00,83.44,97.51,112.47,128.19,144.56,161.51,179.00,197.05,...,234.84,254.62,275.02,296.05,317.74,340.10,363.15,386.93,411.44,436.71
3,0.00,0.00,0.00,69.73,82.93,97.16,112.25,128.06,144.47,161.44,...,196.99,215.60,234.78,254.57,274.97,296.00,317.69,340.05,363.11,386.88
4,0.00,0.00,0.00,0.00,56.87,69.06,82.45,96.85,112.07,127.95,...,161.38,178.88,196.93,215.54,234.72,254.51,274.91,295.94,317.63,340.00


In [46]:
am_call_df = pd.DataFrame(binomial_tree_option.generate_american_call())
am_call_df = am_call_df.round(2)
#am_call_df.replace(0, '', inplace=True)
am_call_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,113.02,128.55,144.79,161.67,179.14,197.17,215.78,234.96,254.74,275.13,...,317.84,340.20,363.25,387.02,411.53,436.80,462.85,489.72,517.42,545.98
1,0.00,97.89,112.73,128.36,144.67,161.58,179.07,197.11,215.72,234.90,...,275.08,296.11,317.79,340.15,363.20,386.97,411.48,436.75,462.81,489.68
2,0.00,0.00,83.44,97.51,112.47,128.19,144.56,161.51,179.00,197.05,...,234.84,254.62,275.02,296.05,317.74,340.10,363.15,386.93,411.44,436.71
3,0.00,0.00,0.00,69.73,82.93,97.16,112.25,128.06,144.47,161.44,...,196.99,215.60,234.78,254.57,274.97,296.00,317.69,340.05,363.11,386.88
4,0.00,0.00,0.00,0.00,56.87,69.06,82.45,96.85,112.07,127.95,...,161.38,178.88,196.93,215.54,234.72,254.51,274.91,295.94,317.63,340.00


In [47]:
print(f"Binomial European Call Value: {binomial_tree_option.european_call_price(): .3f}\nBinomial American Call Value: {binomial_tree_option.american_call_price(): .3f}")

Binomial European Call Value:  113.018
Binomial American Call Value:  113.018


In [48]:
num_steps = 20
initial_guess = 0.5
tolerance = 1e-3
max_iterations = 100

In [49]:
def implied_volatility_call_eu_binomial_tree(call_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, num_steps=num_steps, initial_guess=initial_guess, tolerance=tolerance, max_iterations=max_iterations):
    def binomial_tree_call_price(volatility):
        bt = BinomialOptionPricing(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps)
        return bt.european_call_price()

    # Newton-Raphson method
    vol = initial_guess
    for _ in range(max_iterations):
        option_price = binomial_tree_call_price(vol)
        vega = (binomial_tree_call_price(vol * 1.001) - binomial_tree_call_price(vol)) / (0.001 * vol)

        if abs(option_price - call_price) < tolerance:
            return vol

        vol -= (option_price - call_price) / vega

    return np.nan  # If not found within tolerance

call_price = 125
implied_vol = implied_volatility_call_eu_binomial_tree(call_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity)
rounded_implied_vol = round(float(implied_vol), 3)
print("Implied Volatility European Call (Binomial Tree):", rounded_implied_vol)

Implied Volatility European Call (Binomial Tree): 1.253


In [50]:
def implied_volatility_call_am_binomial_tree(call_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, num_steps=num_steps, initial_guess=initial_guess, tolerance=tolerance, max_iterations=max_iterations):
    def binomial_tree_call_price(volatility):
        bt = BinomialOptionPricing(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps)
        return bt.american_call_price()

    # Newton-Raphson method
    vol = initial_guess
    for _ in range(max_iterations):
        option_price = binomial_tree_call_price(vol)
        vega = (binomial_tree_call_price(vol * 1.001) - binomial_tree_call_price(vol)) / (0.001 * vol)

        if abs(option_price - call_price) < tolerance:
            return vol

        vol -= (option_price - call_price) / vega

    return np.nan  # If not found within tolerance

call_price = 125
implied_vol = implied_volatility_call_eu_binomial_tree(call_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity)
rounded_implied_vol = round(float(implied_vol), 3)
print("Implied Volatility American Call (Binomial Tree):", rounded_implied_vol)

Implied Volatility American Call (Binomial Tree): 1.253


### Put Option

In [51]:
import numpy as np
# Example usage:
spot_price = 516.57   # Current stock price
strike_price = 540.0   # Option strike price
time_to_maturity = difference.days/365  # Time to option expiration in years
volatility = 0.129892 #np.sqrt(0.1)  # Volatility of the underlying stock
risk_free_rate = Risk_Free # Risk-free interest rate
dividend =  dividend_yield_rate
dividend_yield =  risk_free_rate - dividend

num_steps = 20

binomial_tree_option = BinomialOptionPricing(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps)

In [52]:
Asset_Price_df = pd.DataFrame(binomial_tree_option.generate_asset_price())
Asset_Price_df = Asset_Price_df.round(2)
#Asset_Price_df.replace(0, '', inplace=True)
#Asset_Price_df.head(num_steps+1)
Asset_Price_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,516.57,519.91,523.28,526.66,530.07,533.50,536.95,540.43,543.92,547.44,...,554.55,558.14,561.75,565.39,569.05,572.73,576.43,580.16,583.92,587.70
1,0.00,513.25,516.57,519.91,523.28,526.66,530.07,533.50,536.95,540.43,...,547.44,550.99,554.55,558.14,561.75,565.39,569.05,572.73,576.43,580.16
2,0.00,0.00,509.95,513.25,516.57,519.91,523.28,526.66,530.07,533.50,...,540.43,543.92,547.44,550.99,554.55,558.14,561.75,565.39,569.05,572.73
3,0.00,0.00,0.00,506.67,509.95,513.25,516.57,519.91,523.28,526.66,...,533.50,536.95,540.43,543.92,547.44,550.99,554.55,558.14,561.75,565.39
4,0.00,0.00,0.00,0.00,503.41,506.67,509.95,513.25,516.57,519.91,...,526.66,530.07,533.50,536.95,540.43,543.92,547.44,550.99,554.55,558.14


In [53]:
euro_put_df = pd.DataFrame(binomial_tree_option.generate_european_put())
euro_put_df = euro_put_df.round(2)
#euro_put_df.replace(0, '', inplace=True)
euro_put_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,22.88,19.83,16.85,13.98,11.27,8.77,6.52,4.58,2.99,1.77,...,0.39,0.12,0.02,0.00,0.00,0.00,0.00,0.0,0.0,0.0
1,0.00,26.02,22.88,19.78,16.76,13.84,11.07,8.51,6.21,4.24,...,1.45,0.67,0.23,0.04,0.00,0.00,0.00,0.0,0.0,0.0
2,0.00,0.00,29.23,26.05,22.88,19.75,16.67,13.69,10.86,8.23,...,3.86,2.26,1.12,0.42,0.09,0.00,0.00,0.0,0.0,0.0
3,0.00,0.00,0.00,32.50,29.30,26.09,22.90,19.72,16.60,13.55,...,7.92,5.49,3.43,1.83,0.75,0.18,0.00,0.0,0.0,0.0
4,0.00,0.00,0.00,0.00,35.77,32.58,29.37,26.15,22.93,19.72,...,13.42,10.42,7.60,5.06,2.93,1.34,0.37,0.0,0.0,0.0


In [54]:
am_put_df = pd.DataFrame(binomial_tree_option.generate_american_put())
am_put_df = am_put_df.round(2)
#am_put_df.replace(0, '', inplace=True)
am_put_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,23.48,20.28,17.19,14.23,11.45,8.89,6.60,4.63,3.02,1.78,...,0.39,0.12,0.02,0.00,0.00,0.00,0.00,0.0,0.0,0.0
1,0.00,26.75,23.44,20.22,17.08,14.07,11.23,8.62,6.28,4.28,...,1.46,0.67,0.23,0.04,0.00,0.00,0.00,0.0,0.0,0.0
2,0.00,0.00,30.05,26.75,23.43,20.17,16.98,13.91,11.01,8.33,...,3.89,2.28,1.12,0.42,0.09,0.00,0.00,0.0,0.0,0.0
3,0.00,0.00,0.00,33.33,30.05,26.75,23.43,20.12,16.88,13.76,...,8.01,5.54,3.46,1.84,0.76,0.18,0.00,0.0,0.0,0.0
4,0.00,0.00,0.00,0.00,36.59,33.33,30.05,26.75,23.43,20.09,...,13.61,10.54,7.68,5.11,2.95,1.34,0.37,0.0,0.0,0.0


In [55]:
print(f"Binomial European Put Value: {binomial_tree_option.european_put_price(): .3f}\nBinomial American Put Value: {binomial_tree_option.american_put_price(): .3f}")

Binomial European Put Value:  22.885
Binomial American Put Value:  23.476


In [56]:
num_steps = 20
initial_guess = 0.5
tolerance = 1e-3
max_iterations = 100

In [57]:
def implied_volatility_put_am_binomial_tree(call_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, num_steps = num_steps, initial_guess = initial_guess, tolerance = tolerance, max_iterations = max_iterations):
    def binomial_tree_put_price(volatility):
        bt = BinomialOptionPricing(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps)
        return bt.european_put_price()

    # Newton-Raphson method
    vol = initial_guess
    for _ in range(max_iterations):
        option_price = binomial_tree_put_price(vol)
        vega = (binomial_tree_put_price(vol * 1.001) - binomial_tree_put_price(vol)) / (0.001 * vol)
        if abs(option_price - call_price) < tolerance:
            return vol

        vol -= (option_price - call_price) / vega

    return np.nan  # If not found within tolerance

put_price = 25
implied_vol = implied_volatility_put_am_binomial_tree(put_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity)
rounded_implied_vol = round(float(implied_vol), 3)
print("Implied Volatility European Put (Binomial Tree):", rounded_implied_vol)

Implied Volatility European Put (Binomial Tree): 0.218


In [58]:
def implied_volatility_put_am_binomial_tree(call_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, num_steps = num_steps, initial_guess = initial_guess, tolerance = tolerance, max_iterations = max_iterations):
    def binomial_tree_put_price(volatility):
        bt = BinomialOptionPricing(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps)
        return bt.american_put_price()

    # Newton-Raphson method
    vol = initial_guess
    for _ in range(max_iterations):
        option_price = binomial_tree_put_price(vol)
        vega = (binomial_tree_put_price(vol * 1.001) - binomial_tree_put_price(vol)) / (0.001 * vol)
        if abs(option_price - call_price) < tolerance:
            return vol

        vol -= (option_price - call_price) / vega

    return np.nan  # If not found within tolerance

put_price = 25
implied_vol = implied_volatility_put_am_binomial_tree(put_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity)
rounded_implied_vol = round(float(implied_vol), 3)
print("Implied Volatility Americna Put (Binomial Tree):", rounded_implied_vol)

Implied Volatility Americna Put (Binomial Tree): 0.207
